In [1]:
import numpy as np
import pandas as pd
import sys, os, time
main_folder = './../'
if main_folder not in sys.path:
    sys.path.append(main_folder)
from ga import GeneticAlgorithm
from glob       import glob
from tqdm       import tqdm

import dirs
from utils      import get_solution, ProgressBar, write_log
import matplotlib.pyplot as plt 

progbar = ProgressBar(elapsed_time=True)

### Parameters

In [2]:
# Competition Parameters
funcId      = 1
dim         = 10
numRuns     = 51
targetError = 1e-8

# Algorithm Parameters
pop_size = 100
percMaxEvals = 1
maxEvals = 'auto'
maxGenerations = None
fitness_clusters = None
crossover = 'onePointCrossOver'
mutation = 'uniform'
elitism = 1
opposition = False
input_data_filepath = './../input_data'
active = False

In [3]:
maxEvals = maxEvals if percMaxEvals is None else percMaxEvals*10000*dim
run_tag = '[' + '_'.join([str(funcId), 
                          str(dim), 
                          str(numRuns),
                          str(pop_size), 
                          str(percMaxEvals),                                                                               
                          str(opposition),
                          str(fitness_clusters)
                         ])  + ']'
run_tag

'[1_10_51_100_1_False_None]'

In [4]:
alg = GeneticAlgorithm(dim=dim, func_id=funcId, pop_size=pop_size, crossover=crossover, 
        opposition=opposition, mutation=mutation, elitism=elitism, fitness_clusters=fitness_clusters)

output_filepath = os.path.join(dirs.results, str(alg))
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_filepath = output_filepath + '/' + run_tag

print ('Writing log at ', output_filepath + 'log.tsv')
write_log(filepath=output_filepath + 'log.tsv', 
          mode="w+", 
          text='[{}]\trun\telapsedTime\tsuccessRate\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())))  

Writing log at  ./results/GA/uniform/1/one/[1_10_51_100_1_False_None]log.tsv


In [6]:
hist = pd.DataFrame()
arr_runs = range(numRuns)
solution = get_solution(funcId, dim, input_data_filepath=input_data_filepath)
successRate = 0
for idx_run in arr_runs:
    progbar.update_progress(idx_run/float(len(arr_runs)))
    
    # Differential Evolution
    alg = GeneticAlgorithm(dim=dim, func_id=funcId, pop_size=pop_size, crossover=crossover, 
        opposition=opposition, mutation=mutation, elitism=elitism, fitness_clusters=fitness_clusters)
    
    errorHist, fitnessHist = alg.optimize(target=solution, max_f_evals=maxEvals, target_error=targetError, verbose=True)        
    bestError = errorHist.iloc[-1,:].min()
    errorHist["run"] = np.ones(errorHist.shape[0], dtype=int)*idx_run
    hist = pd.concat([hist, errorHist], ignore_index=False)    
    if bestError <= targetError:
        print ("Target error achieved with error {}".format(bestError))
        successRate += 1
    
    write_log(filepath=output_filepath + 'log.tsv', 
              text='[{}]\t{}/{}\t{}\t{}\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()), 
                                              idx_run+1,
                                              len(arr_runs),
                                              progbar.get_elapsed_time(),
                                              successRate)
             )
    
hist.index.name = 'generation'
hist.to_csv(output_filepath + 'hist.tsv', sep='\t')
progbar.update_progress(1)

[0:00:00][----------] 0.00% Optimization ended due to max fitness evals (max = 100000, curr = 53414)

#Generations:	46588
#FitnessEvals:	53414
Mean Fitness:	35408749.8460
Best Fitness:	35408749.8460
Solution:	100.0000
Diff:		35408649.8460
[0:22:01][----------] 1.96% Optimization ended due to max fitness evals (max = 100000, curr = 52316)

#Generations:	47686
#FitnessEvals:	52316
Mean Fitness:	67381964.5470
Best Fitness:	67381964.5470
Solution:	100.0000
Diff:		67381864.5470
[0:42:17][----------] 3.92% Optimization ended due to max fitness evals (max = 100000, curr = 52130)

#Generations:	47872
#FitnessEvals:	52130
Mean Fitness:	42858913.4518
Best Fitness:	42858913.4518
Solution:	100.0000
Diff:		42858813.4518
[1:02:43][#---------] 5.88% Optimization ended due to max fitness evals (max = 100000, curr = 52984)

#Generations:	47018
#FitnessEvals:	52984
Mean Fitness:	56247028.9432
Best Fitness:	56247028.9432
Solution:	100.0000
Diff:		56246928.9432
[1:22:31][#---------] 7.84% Optimization end

[11:08:45][#######---] 66.67% Optimization ended due to max fitness evals (max = 100000, curr = 51196)

#Generations:	48806
#FitnessEvals:	51196
Mean Fitness:	85559313.6377
Best Fitness:	85559313.6377
Solution:	100.0000
Diff:		85559213.6377
[11:29:46][#######---] 68.63% Optimization ended due to max fitness evals (max = 100000, curr = 51034)

#Generations:	48968
#FitnessEvals:	51034
Mean Fitness:	32353571.1498
Best Fitness:	32353571.1498
Solution:	100.0000
Diff:		32353471.1498
[11:53:49][#######---] 70.59% Optimization ended due to max fitness evals (max = 100000, curr = 55009)

#Generations:	44993
#FitnessEvals:	55009
Mean Fitness:	71933588.9759
Best Fitness:	71933588.9759
Solution:	100.0000
Diff:		71933488.9759
[12:16:30][#######---] 72.55% Optimization ended due to max fitness evals (max = 100000, curr = 52767)

#Generations:	47235
#FitnessEvals:	52767
Mean Fitness:	61372664.2145
Best Fitness:	61372664.2145
Solution:	100.0000
Diff:		61372564.2145
[12:36:27][#######---] 74.51% Optimi

In [7]:
print ("Elapsed Time: ", progbar.get_elapsed_time())

Elapsed Time:  16:41:25


In [8]:
errorHist.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,run
generation,,,,,,,,,,,,,,,,,,,,,
42261,7.149974e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
42262,7.149974e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
42263,7.149974e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
42264,7.149974e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
42265,7.149974e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50


In [9]:
arr_error = list()
arr_best_error = list()
for run in hist['run'].unique():
    df_run = hist[hist['run'] == run].drop(['run'], axis=1) 
    bestError = df_run.iloc[-1,:].min()    
    arr_best_error.append(bestError)    
    print ('Best error for run {}: {}'.format(run, bestError))
    arr_error.append(np.array(df_run.min(axis=1)))        
arr_error = np.array(arr_error)



Best error for run 0: 35408649.846025616
Best error for run 1: 67381864.54696338
Best error for run 2: 42858813.4518164
Best error for run 3: 56246928.94317704
Best error for run 4: 105155751.00372252
Best error for run 5: 46463293.50310828
Best error for run 6: 85601765.81124046
Best error for run 7: 42216178.85214284
Best error for run 8: 48525390.03699241
Best error for run 9: 192044419.51112887
Best error for run 10: 47520287.92642329
Best error for run 11: 77366184.59134895
Best error for run 12: 64469369.8540019
Best error for run 13: 104889227.01093982
Best error for run 14: 53132184.801995635
Best error for run 15: 77602473.34900334
Best error for run 16: 65145355.47996774
Best error for run 17: 74575994.08716595
Best error for run 18: 103931263.812049
Best error for run 19: 63330528.062037975
Best error for run 20: 87219881.70227256
Best error for run 21: 80712341.19908841
Best error for run 22: 45109180.63753484
Best error for run 23: 133820606.51770414
Best error for run 24:

In [10]:
df_sumup = pd.DataFrame(columns=['algorithm', '#Fc', 'D', 'percFES'] + ['run_' + str(i) for i in np.arange(1,numRuns+1,1)] + ['mean', 'elapsed_time', 'evals'])
df_sumup.loc[df_sumup.shape[0]] = [str(alg), funcId, dim, percMaxEvals] + arr_best_error + [np.mean(arr_best_error), progbar.get_elapsed_time(), alg.fitnessEvals]
print ('Saving df_sumup to ', output_filepath + 'df_sumup.tsv')
df_sumup.to_csv(output_filepath + 'df_sumup.tsv', sep='\t')
df_sumup

Saving df_sumup to  ./results/GA/uniform/1/one/[1_10_51_100_1_False_None]df_sumup.tsv


,algorithm,#Fc,D,percFES,run_1,run_2,run_3,run_4,run_5,run_6,...,run_45,run_46,run_47,run_48,run_49,run_50,run_51,mean,elapsed_time,evals
0,GA/uniform/1/one,1,10,1,3.540865e+07,6.738186e+07,4.285881e+07,5.624693e+07,1.051558e+08,4.646329e+07,...,6.621235e+07,2.974990e+07,1.410150e+08,1.079939e+08,5.179089e+07,5.989625e+07,7.149974e+07,7.860324e+07,16:41:25,57736


In [ ]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10))
# label = 'Success Rate: {}%\nMin Error Fitness: {}'.format(100*(successRate/float(numRuns)), arr_error.min())
# # label = ''
# ax.errorbar(np.arange(alg.generations), arr_error.mean(axis=0), yerr=arr_error.std(axis=0), label=label)
# ax2 = ax.twinx()
# ax2.plot(np.arange(alg.generations), 20*np.log(arr_error.mean(axis=0)), label='Mean Min Error (dB)', color='red')
# ax2.set_ylabel('Mean Minimum Error (dB)')

# ax.grid(True)
# ax.set_ylabel('Mean Minimum Error')
# ax.legend()
# ax.set_xlabel('Generation')
# ax.set_title('Error Fitness Function {} (Solution = {})'.format(funcId, solution))
# plt.savefig(output_filepath + 'mean_error_evolution.jpeg', bbox_inches = 'tight')